In [1]:
import numpy as numpy
from matplotlib import pyplot as matplot
%matplotlib inline
import pandas as pandas
import math
from sklearn import preprocessing
from sklearn import svm
import cv2
import os
import ntpath
from sklearn.metrics.pairwise import cosine_similarity
import fnmatch
import glob
import math
from math import sqrt,log
from cmath import sqrt

data = []
Filename = []
Person = []
Testdata = []
TestFilename = []
TestPerson = []
angles = []
Len = []
Rho = []


def crop_image(img,tol=0):
    mask = img>tol
    return img[numpy.ix_(mask.any(1),mask.any(0))]
def thinning(img):
    #img = cropimage
    size = numpy.size(img)
    skel = numpy.zeros(img.shape,numpy.uint8)

    ret,img = cv2.threshold(img,127,255,0)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    done = False

    while( not done):
        eroded = cv2.erode(img,element)
        temp = cv2.dilate(eroded,element)
        temp = cv2.subtract(img,temp)
        skel = cv2.bitwise_or(skel,temp)
        img = eroded.copy()

        zeros = size - cv2.countNonZero(img)
        if zeros==size:
            done = True

  #  plt.imshow(skel,'gray')
   # plt.show()
    return skel
def coords(thinnedimage):
    rows,cols=thinnedimage.shape
    img_topleft=thinnedimage[0:int(rows/2),0:int(cols/2)]
    img_topright=thinnedimage[int(rows/2)+1:rows,0:int(cols/2)]
    img_bottomleft=thinnedimage[0:int(rows/2),int(cols/2)+1:cols]
    img_bottomright=thinnedimage[int(rows/2)+1:rows,int(cols/2)+1:cols]
    
    topleft_x,topleft_y=COG(img_topleft)
    topright_x,topright_y=COG(img_topright)
    bottomleft_x,bottomleft_y=COG(img_bottomleft)
    bottomright_x,bottomright_y=COG(img_bottomright)

    return topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y

def COG(img):
    x_cor=0
    xrun_sum=0
    y_cor=0
    yrun_sum=0
    #print(img.shape)
    for i in range(img.shape[0]):
        x_cor+=sum(img[i])*i/255
        xrun_sum+=sum(img[i])/255

    for i in range(img.shape[1]):
        y_cor+=sum(img[:,i])*i/255
        yrun_sum+=sum(img[:,i])/255
        #print(img.shape[1]) 
        if yrun_sum==0:
            x_pos=0
        else:
            x_pos=y_cor/(yrun_sum)
        if xrun_sum==0:
            y_pos=0
        else:
            y_pos=x_cor/(xrun_sum)
        
   # print(x_pos)
  #  print(y_pos)
    
    return (x_pos/img.shape[1],y_pos/img.shape[0])

def tan_i(x):
    #print(x)
    if x[0]==0:
        return 90
    return math.degrees(math.atan(x[1]/x[0]))

def tan(thinnedimage):
    rows,cols=thinnedimage.shape
    
    img_tl1=thinnedimage[0:int(rows/2),0:int(cols/4)]
    img_tl2=thinnedimage[0:int(rows/2),int(cols/4)+1:int(cols/2)]
    
    img_tr1=thinnedimage[0:int(rows/2),int(cols/2)+1:int(0.75*cols)]
    img_tr2=thinnedimage[0:int(rows/2),int(0.75*cols)+1:cols]
    
    img_bl1=thinnedimage[int(rows/2)+1:rows,0:int(cols/4)]
    img_bl2=thinnedimage[int(rows/2)+1:rows,int(cols/4)+1:int(cols/2)]
    
    img_br1=thinnedimage[int(rows/2)+1:rows,int(cols/2)+1:int(0.75*cols)]
    img_br2=thinnedimage[int(rows/2)+1:rows,int(0.75*cols)+1:cols]
    

    #plt.imshow(timg,'gray')
    #plt.show()
    
    tl1=tan_i(COG(img_tl1))
    tl2=tan_i(COG(img_tl2))
    tr1=tan_i(COG(img_tr1))
    tr2=tan_i(COG(img_tr2))
    bl1=tan_i(COG(img_bl1))
    bl2=tan_i(COG(img_bl2))
    br1=tan_i(COG(img_br1))
    br2=tan_i(COG(img_br2))
    
    #plt.imshow(img_br1,'gray')
    #plt.show()
    #print(COG(img_br1))
    return tl1,tl2,tr1,tr2,bl1,bl2,br1,br2

def leng_rho(img):
    kernel = numpy.ones((2,2), numpy.uint8)
    img_dilation = cv2.dilate(img, kernel, iterations=2)
    img_erosion = cv2.erode(img, kernel, iterations=4)
    edges = cv2.Canny(img_erosion,cv2.THRESH_BINARY+cv2.THRESH_OTSU,550,apertureSize = 3)
    minLineLength=100
    maxLineGap=10
    threshold=20
    image=edges
    lines = cv2.HoughLinesP(image=img,rho=1,theta=numpy.pi/180, threshold=threshold,lines=numpy.array([]), minLineLength=minLineLength,maxLineGap=maxLineGap)
    for x1,y1,x2,y2 in lines[0]:
        theta = math.atan(float(y2 - y1) / (x2 - x1))
        Leng = sqrt((x2-x1)^2 + (y2-y1)^2)
        Rho = (x1 * y2 - x2 * y1) / Leng
        angles.append({'theta': theta, 'p1': (x1, y1), 'p2': (x2, y2)})
    #for i in range(len(angles)):
     #   for j in range(i + 1, len(angles)):
      #      delta = abs(angles[i]['theta'] - angles[j]['theta'])               
    a,b,c = lines.shape
    theta = round(theta,4)
    Rho = round(abs(Rho),0)
    Leng = round(abs(Leng),0)
    return a,b,c,theta,Rho,Leng


def createfingerprint_Test(image):
    aspect_ratio = 1.0*(image.shape[1]/image.shape[0])
    ret,thresh1 = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
    ret,thresh2 = cv2.threshold(image,127,255,cv2.THRESH_BINARY_INV)
    ret,thresh3 = cv2.threshold(image,127,255,cv2.THRESH_TRUNC)
    ret,thresh4 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO)
    ret,thresh5 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO_INV)
    a,b,c,theta,Rho,Leng = leng_rho(image)
    blur = cv2.GaussianBlur(thresh1,(5,5),0)
    ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image=numpy.invert(th3)
    
    cropimage=crop_image(image,tol=0)
    area=cv2.countNonZero(cropimage)
    area=cv2.countNonZero(cropimage)/(cropimage.shape[0]*cropimage.shape[1])

    img1=numpy.invert(cropimage)
    connectcomp=cv2.connectedComponents(img1)[0]

    thinnedimage=thinning(cropimage)
    #Thinning the image!

    topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y=coords(thinnedimage)
    tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y = tan(thinnedimage)
    
        #Extracting features

    b=pandas.Series([area,connectcomp,topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y,tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y,aspect_ratio,a,b,c,theta,Rho,Leng],index=
                    ["Norm_area","connected_comps","topleft_x","topleft_y","topright_x","topright_y","bottomleft_x","bottomleft_y","bottomright_x","bottomright_y","tan_topleft_x", "tan_topleft_y", "tan_topright_x", "tan_topright_y", "tan_bottomleft_x", "tan_bottomleft_y", "tan_bottomright_x", "tan_bottomright_y","Aspect_Ratio","a","b","c","theta","Rho","Leng"])
    Testdata.append(b)    
    

def create_test_dataset(x):
    os.chdir(Testdir)
    path = glob.glob(Testdir+File)[0]
    fn = File
    tn = Person_Name
    TestFilename.append(fn)
    TestPerson.append(tn)
    image = cv2.imread(path,0)
    createfingerprint_Test(image)
    Test_Signature_Table=pandas.DataFrame(Testdata)
    Test_Cosine = pandas.DataFrame(cosine_similarity(Test_Signature_Table), index = None)
            
            

    Test_Signature_Table['Filename'] = TestFilename
    Test_Cosine['Filename'] = TestFilename
    Trainingdatacol = Test_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Test_Signature_Table = Test_Signature_Table[Trainingdatacol]
    Test_Signature_Table['Person'] = TestPerson
    Trainingdatacol = Test_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Test_Signature_Table = Test_Signature_Table[Trainingdatacol]
    Test_Signature_Table['Category'] = "Test"
    Cosinecolumns = Test_Cosine.columns.tolist()
    Cosinecolumns = Cosinecolumns[-1:]+Cosinecolumns[:-1]
    Test_Cosine = Test_Cosine[Cosinecolumns]
    #Original_Signature_Table = pandas.concat([Original_Signature_Table,Cosine],axis = 1)
    Test_Signature_Table.to_csv('Test.csv', index = False)
    Test_Cosine.to_csv('Cosine_Test.csv', index = False)
    Summary_Table = pandas.DataFrame(Test_Signature_Table.describe()).T
    Summary_Table.to_csv('Test_Summary.csv', index = False)
    Test_Signature_Table
    Test_Cosine
    print('Preprocessing of Test Data Completed')
    
def zscore(Test,Mean,Std):
    Output_Table = (Test.loc[0]-Mean)/Std
    return(Output_Table)

In [10]:
Testdir = 'C:\\Siddhartha\\Confidential Documents\\Data Science\\Signature Verification\\Testing\\'
File = "Test.PNG"
Person_Name = 1

In [11]:
create_test_dataset(1)

Preprocessing of Test Data Completed


In [14]:
Genuinedir = 'C:\Siddhartha\Confidential Documents\Data Science\Signature Verification\Testing'
os.chdir(Genuinedir)
Original_Signature_Table = pandas.read_csv("Train.csv")
os.chdir(Testdir)
Test_Signature_Table = pandas.read_csv("Test.csv")
Test_1by1 = Test_Signature_Table
del Test_Signature_Table["Category"]

In [15]:
Original_1by1 = Original_Signature_Table.loc[Original_Signature_Table["Person"] == Person_Name]
Compare = pandas.merge(Test_1by1,Original_1by1,on = ["Norm_area","connected_comps","topleft_x","topleft_y","topright_x","topright_y","bottomleft_x","bottomleft_y","bottomright_x","bottomright_y","tan_topleft_x", "tan_topleft_y", "tan_topright_x", "tan_topright_y", "tan_bottomleft_x", "tan_bottomleft_y", "tan_bottomright_x", "tan_bottomright_y","Aspect_Ratio"], how = 'inner')
Count = Compare.count()[1]
if Count != 0:
    print("Match Found in Original Signature")
else:
    print("Match not found in Original Signature, Checking using alternative method")

Match Found in Original Signature


In [16]:
Mean_Original_Signature = pandas.DataFrame(Original_Signature_Table.groupby('Person').mean().reset_index())
Stddev_Original_Signature = pandas.DataFrame(Original_Signature_Table.groupby('Person').std().reset_index())
a = Test_Signature_Table.index[0]
Test = pandas.DataFrame(Test_Signature_Table.iloc[0:1,2:])
Mean = pandas.DataFrame(Mean_Original_Signature.loc[Mean_Original_Signature["Person"] == Person_Name].iloc[0:1,1:])
Std = pandas.DataFrame(Stddev_Original_Signature.loc[Stddev_Original_Signature["Person"] == Person_Name].iloc[0:1,1:])
Compared = zscore(Test,Mean,Std)
T = (pandas.DataFrame((((Compared >= -1) & (Compared<=1) & (Compared != 0)).loc[Compared.index[0]]).value_counts(sort = True)).reset_index())
T.columns = ["Match_Flag","Count"]
Matched_Rate = (round(((T.loc[T["Match_Flag"] == True]["Count"]).reset_index(drop = True)[0])/(T["Count"].sum()),4))*100
print("Matching% = ", round(Matched_Rate,2))

Matching% =  92.0
